In [1]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# On charge le dataset directement depuis Keras.
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data(path="mnist.npz")

# On reformate les images pour leur donner la forme (28, 28, 1) qui est la manière standard de représenter une image en ML.
# On les ramène aussi à des valeurs entre 0 et 1 car les réseaux de neurones fonctionnent mieux sur des valeurs proches de 0.
X_train, X_test = X_train[..., np.newaxis] / 255.0, X_test[..., np.newaxis] / 255.0

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [8]:
print(X_train.shape, X_test.shape)

(48000, 28, 28, 1) (10000, 28, 28, 1)


In [3]:
# Construisez votre modèle ici
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(8, 3, activation='relu', padding="same"),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Conv2D(8, 3, activation='relu', padding="same"),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Conv2D(1, 3, activation='relu', padding="same"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='linear'),
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [4]:
# Affichage de la structure du modèle
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 8)         80        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 8)         584       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 1)           73        
_________________________________________________________________
flatten (Flatten)            (None, 49)                0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

In [5]:
# Entraînez votre modèle ici
model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)],
)

Epoch 1/10
1500/1500 [==============================] - 15s 10ms/step - loss: 0.4120 - accuracy: 0.8694 - val_loss: 0.1893 - val_accuracy: 0.9412
Epoch 2/10
1500/1500 [==============================] - 14s 10ms/step - loss: 0.1676 - accuracy: 0.9467 - val_loss: 0.1416 - val_accuracy: 0.9553
Epoch 3/10
1500/1500 [==============================] - 14s 10ms/step - loss: 0.1275 - accuracy: 0.9608 - val_loss: 0.1238 - val_accuracy: 0.9604
Epoch 4/10
1500/1500 [==============================] - 15s 10ms/step - loss: 0.1047 - accuracy: 0.9667 - val_loss: 0.0952 - val_accuracy: 0.9697
Epoch 5/10
1500/1500 [==============================] - 17s 11ms/step - loss: 0.0920 - accuracy: 0.9705 - val_loss: 0.1155 - val_accuracy: 0.9642


In [6]:
# Evaluation du modèle sur le test set
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 5ms/step - loss: 0.0962 - accuracy: 0.9674


[0.09622029215097427, 0.9674000144004822]

In [7]:
# Sauvegarde du modèle
model.save("models/mnist_model.h5")